## Распознование текста из микрофона

In [ ]:
sudo apt-get install portaudio19-dev

In [1]:
!environment/bin/pip install SpeechRecognition
!environment/bin/pip install PyAudio

In [2]:
import speech_recognition as sr

In [3]:
recognizer = sr.Recognizer()

In [4]:
recognizer.listen??

In [4]:
with sr.Microphone() as micro:
    print(1)
    audio = recognizer.listen(micro)
    print(2)

1
2


In [54]:
with open('data/microphone-results.wav', "wb") as f:
    f.write(audio.get_wav_data())

In [5]:
recognizer.recognize_google(audio, language='en-EN')

'speak speak speak speak speak speak'

## Распознование текста из аудиофайла

In [31]:
recognizer = sr.Recognizer()

In [58]:
with sr.AudioFile('data/microphone-results.wav') as source:
    audio1 = recognizer.record(source)

In [59]:
recognizer.recognize_google(audio1, language='en-EN')

'this is a sound detection chick 111 detection check'

## Text topic extraction

In [36]:
!py3_env/bin/pip install spacy
!py3_env/bin/pip install pymorphy2

In [37]:
import spacy

In [39]:
# spacy.load('ru')

In [60]:
from spacy.lang.en import English

In [61]:
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [63]:
text = recognizer.recognize_google(audio, language='en-EN')
tokenize(text)

['this', 'is', 'a', 'sound', 'detection', 'chick', '111', 'detection', 'check']

In [3]:
!environment/bin/pip install nltk

In [4]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [5]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [16]:
get_lemma("их")

'их'

In [78]:
for token in tokenize(text):
    print(get_lemma(token), get_lemma2(token))

this this
be is
a a
sound sound
detection detection
chick chick
111 111
detection detection
check check


In [11]:
nltk.download('stopwords')
# en_stop = set(nltk.corpus.stopwords.words('english'))
en_stop = set(nltk.corpus.stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /home/anton-
[nltk_data]     taleckij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
list(en_stop)[:5]

['со', 'при', 'чтобы', 'может', 'без']

In [90]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    return [get_lemma(token) for token in tokens 
            if len(token) > 4 and token not in en_stop]

In [91]:
text_data = [prepare_text_for_lda(text)]
text_data

[['sound', 'detection', 'chick', 'detection', 'check']]

In [82]:
!py3_env/bin/pip install gensim

In [86]:
from gensim import corpora

dictionary = corpora.Dictionary(text_data)
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in text_data]
print(corpus)

Dictionary(4 unique tokens: ['check', 'chick', 'detection', 'sound'])
[[(0, 1), (1, 1), (2, 2), (3, 1)]]


In [89]:
dictionary.doc2bow?? # document to bag of words

In [88]:
import gensim

NUM_TOPICS = 2
NUM_WORDS = 3

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = ldamodel.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.251*"detection" + 0.250*"check" + 0.250*"sound"')
(1, '0.359*"detection" + 0.214*"chick" + 0.214*"sound"')


## загрузка аудиофайла из youtube

In [31]:
!py3_env/bin/pip install youtube_dl
!py3_env/bin/pip install oauth2client
!py3_env/bin/pip install pydub

In [14]:
import youtube_dl

ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192'
    }],
    'outtmpl': '1',
    'prefer_ffmpeg': True,
    'keepvideo': False
}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=J5O5iRpLXKY&t=637s'])

[youtube] J5O5iRpLXKY: Downloading webpage
[youtube] J5O5iRpLXKY: Downloading video info webpage
[download] Destination: 1
[download] 100% of 15.11MiB in 00:13.00KiB/s ETA 00:000
[ffmpeg] Destination: wav
Deleting original file 1 (pass -k to keep)


In [15]:
from pydub import AudioSegment
t1 = 110 * 1000 #Works in milliseconds
t2 = 200 * 1000
newAudio = AudioSegment.from_wav("data/1.wav")
newAudio = newAudio[t1:t2]
newAudio.export('data/3.wav', format="wav")

<_io.BufferedRandom name='data/3.wav'>

In [57]:
recognizer = sr.Recognizer()
with sr.AudioFile('data/3.wav') as source:
    audio1 = recognizer.record(source)
recognizer.recognize_google(audio1, language='ru_RU')

'чтобы после очень крутой 18 года попробовать себя на западе А это произошло наверное летом летом 18 летом 18 года Так как только артист увидел офсет увидел эту ссылку Он резко отправил своим продюсером и сказал у меня переписку показывали потом эти же продюсер искал это моё секретное оружие Давайте звать парня и потихоньку очень медленно начинался запускается процесс То есть как бы это ни было моментально там о Попов и мы уже Я уже тут В начале обсуждал Ну поскольку у них стартовал тур совместно с дрейком не улавливает of цитата 11 чуваков из группы migos когда архитектуре это всегда любой клип запись альбома идёт очень медленно потому что они ежедневно просто путешествую фотосессии концерты в каждом городе по два по три концерта все солдаут были и наш тоже клип он сильно двигался медленно но ближе наверное в октябре уже нам Мне позвонили сказали всё Поехали Давай всё снимать придумывай идею У артиста из Кто такие заметную столько-то заметок что он хочет видеть и мы сейчас я с этим св

## Analyse podcast topics with wor2vec

In [15]:
import nltk
from gensim.models import Word2Vec, ldamodel

NUM_TOPICS = 10
NUM_WORDS = 5

In [1]:
with open('data/podcast.txt', 'r+') as f:
    text = f.read()
text[100:200]

'й как нервничал тогда секс это правда что вы гений с юрий будет дуть дуть будет юрий мы живем в очен'

In [4]:
tokens = text.split()
print(len(tokens))
en_stop = set(nltk.corpus.stopwords.words('russian'))
filtered_tokens = [token for token in tokens if token not in en_stop]
print(len(filtered_tokens))

6100
3664


In [6]:
model = Word2Vec([filtered_tokens], size=150, window=10, min_count=1, workers=4)

In [7]:
model.wv.vocab

{'дудь': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642cf8>,
 'агент': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642da0>,
 'фсб': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642e10>,
 'гебельс': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642dd8>,
 'жив': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642d68>,
 'застрелился': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642d30>,
 'ещё': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642cc0>,
 'тошнит': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642c50>,
 'нервничай': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642e48>,
 'нервничал': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642e80>,
 'секс': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642ef0>,
 'это': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642f60>,
 'правда': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642f28>,
 'гений': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642eb8>,
 'юрий': <gensim.models.keyedvectors.Vocab at 0x7fd0dd642f98>,
 'дуть': <gensim.models.keyedvect

In [8]:
word_vectors = [model.wv[token] for token in filtered_tokens if token in model.wv.vocab]
word_vectors[:5]

[array([-1.5393798e-03, -1.8648828e-03,  3.1013859e-03, -7.3632429e-04,
        -1.7062041e-03,  1.1556461e-03,  8.2035353e-05,  6.5666571e-04,
         8.7340263e-04, -1.8215020e-03, -9.5723383e-04,  2.2996194e-03,
        -4.5136080e-04, -9.7488184e-05, -9.9431309e-05,  5.0267251e-04,
         3.8772196e-04, -3.1900681e-03,  1.9965349e-03,  2.2073085e-03,
         2.8898031e-03,  2.8708295e-03,  4.4204353e-05, -1.8190990e-03,
        -2.8391174e-04, -3.0403845e-03, -2.7698863e-03,  1.6636519e-03,
        -1.0590102e-03, -9.9975686e-04, -1.2885566e-03,  1.9513068e-03,
        -1.4244590e-04, -1.3696639e-03, -3.1030069e-03,  6.0389115e-04,
        -2.6616228e-03,  2.8581875e-03,  2.6598047e-03,  2.5387087e-03,
        -3.1844210e-03, -6.7774788e-04, -2.0922665e-03,  3.3533953e-03,
         1.2421599e-03, -2.6313242e-05, -9.1279950e-04,  2.9466962e-03,
        -9.3220558e-04,  3.0868663e-03, -1.0448812e-03, -7.0689461e-04,
        -4.6905532e-04, -1.8972972e-03, -2.2976499e-03,  2.84611

In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=NUM_TOPICS, random_state=0).fit(word_vectors)

In [10]:
from collections import Counter
clusters_counter = Counter(kmeans.predict(word_vectors))
clusters_counter

Counter({9: 665,
         1: 571,
         4: 443,
         3: 855,
         6: 170,
         2: 599,
         8: 123,
         0: 127,
         5: 58,
         7: 53})

In [11]:
for i, cluster in enumerate(sorted(clusters_counter, 
                                   key=lambda cluster: clusters_counter[cluster], reverse=True)):
    cluster_vector = kmeans.cluster_centers_[cluster]
    topic = model.wv.similar_by_vector(cluster_vector, topn=NUM_WORDS, restrict_vocab=None)
    print('TOPIC_' + str(i+1), *[word[0] for word in topic])

TOPIC_1 мира выборах крым кажется говорить
TOPIC_2 думаю сегодняшний человек хочу такие
TOPIC_3 украины территории понимание страх территорию
TOPIC_4 году вопрос знаю людей первое
TOPIC_5 люди который чемпионат фсб сказали
TOPIC_6 очень почему украине сказал вообще
TOPIC_7 правда сказать появились отличается гостей
TOPIC_8 это знаете лично оставался украины
TOPIC_9 которые россии такие настойчивой чемпионат
TOPIC_10 всё российской фамилии приехал овладеть


In [16]:
from gensim import corpora

dictionary = corpora.Dictionary([tokens])
# print(dictionary)
corpus = [dictionary.doc2bow(text) for text in [tokens]]
# print(corpus)


lda_model = ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

topics = lda_model.print_topics(num_words=NUM_WORDS)
for topic in topics:
    print(topic)

(0, '0.001*"я" + 0.001*"не" + 0.001*"и" + 0.001*"это" + 0.001*"в"')
(1, '0.028*"и" + 0.027*"я" + 0.027*"не" + 0.026*"в" + 0.022*"что"')
(2, '0.001*"и" + 0.001*"не" + 0.001*"я" + 0.001*"в" + 0.001*"это"')
(3, '0.001*"я" + 0.001*"что" + 0.001*"и" + 0.001*"в" + 0.001*"не"')
(4, '0.001*"и" + 0.001*"не" + 0.001*"в" + 0.001*"я" + 0.001*"что"')
(5, '0.001*"и" + 0.001*"не" + 0.001*"я" + 0.001*"что" + 0.001*"в"')
(6, '0.001*"я" + 0.001*"не" + 0.001*"в" + 0.001*"что" + 0.001*"и"')
(7, '0.001*"не" + 0.001*"и" + 0.001*"я" + 0.001*"в" + 0.001*"что"')
(8, '0.001*"не" + 0.001*"в" + 0.001*"и" + 0.001*"я" + 0.001*"что"')
(9, '0.001*"и" + 0.001*"я" + 0.001*"это" + 0.001*"в" + 0.001*"что"')
